In [1]:
# you should run this demo from kaggle, becasue 
# 1> this demo requires a GPU, kaggle provide free GPU
# 2> colab also provide free GPU, however, colab bans interactive widgets like ipywidgets

import os
os.chdir('/kaggle/input/demo/demo/')
os.getcwd()

'/kaggle/input/demo/demo'

In [2]:
import pickle
import numpy as np
import sys
import tensorflow as tf
import PIL
import io
import ipywidgets

In [3]:
# read in feature axis of latent space
laten_feature_axis_path = './feature_axis.pkl'
with open(laten_feature_axis_path, 'rb') as f:
    feature_axis_name = pickle.load(f)

feature_axis, feature_name = feature_axis_name['direction'], feature_axis_name['name']
feature_axis = feature_axis
feature_num = 40

feature_axis = feature_axis * np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,-1, 1,-1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,-1])[None, :]

In [4]:
feature_axis.shape

(512, 40)

In [5]:
# read in the pre-trained pg-GAN model
pg_gan_code_path = './pggan'
pg_gan_data_path = './karras2018iclr-celebahq-1024x1024.pkl'
sys.path.append(pg_gan_code_path)

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
with open(pg_gan_data_path, 'rb') as file:
    G, D, Gs = pickle.load(file)
z_dimension = 512

In [6]:
# some function to perform Gram-Schmidt process on feature axis matrix
def arc_sigmoid(arr):
    arr_ = np.clip(arr, 10e-324, 1 - 10e-17)
    return np.log(arr_ / (1-arr_))

def arc_tanh(arr):
    arr_ = np.clip(arr, np.tanh(-19.06), np.tanh(19.06))
    return np.arctanh(arr_)    

def normalize(vec):
    return vec / np.linalg.norm(vec)

def Gram_Schmidt(mat_original):
    mat = mat_original.astype(np.float64) + 0
    feature_num = mat.shape[1]
    for i in range(feature_num):
        intermediate = 0
        for j in range(i):
            intermediate += (mat[:, i].dot(mat[:, j])) * mat[:, j]
        mat[:, i] -= intermediate
        mat[:, i] = normalize(mat[:, i])
    return mat

def test_is_orthogonal(arr):
    is_orthogonal = True
    k = 0.0
    for j in range(arr.shape[1]-1):
        k += abs(arr[:,j].dot(arr[:,j+1]))
        print(arr[:,j].dot(arr[:,j+1]))
    return k

In [7]:
# some function to generate a new face image generated by pg-GAN model
def img_process(x):
    img = PIL.Image.fromarray(x)
    imgBIO = io.BytesIO()
    img.save(imgBIO, format='PNG')
    imgB = imgBIO.getvalue()
    return imgB

def generate_img(z, Gs):
    z = z[None, :]
    img = Gs.run(z, np.zeros([z.shape[0], 0]))
    img = np.clip(np.rint((img + 1.0) / 2.0 * 255.0), 0.0, 255.0).astype(np.uint8).transpose(0, 2, 3, 1) 
    return img[0]

In [8]:
# interactive demo
z = np.random.randn(z_dimension)
x = generate_img(z=z, Gs=Gs)

img_widget = ipywidgets.widgets.Image(value=img_process(x), fromat='png', width=512, height=512, layout=ipywidgets.Layout(height='512px', width='512px'))

class Callback(object):
    counter = 0
    def __init__(self):
        self.z = z
        self.feature_axis = Gram_Schmidt(feature_axis)

    def generate_new_img(self, event):
        self.z = np.random.randn(z_dimension)
        self.set_new_img()

    def move_along_axis(self, event, feature_index, step_size):
        self.z += self.feature_axis[:, feature_index] * step_size
        self.set_new_img()
    
    def set_new_img(self):        
        x = generate_img(z=self.z, Gs=Gs)
        img_widget.value = img_process(x)

interactive_callback = Callback()

def modify_feature_widgets(feature_index, width=128, height=40):
    height_1 = '{:.0f}px'.format(height/2)
    width_1 = '{:.0f}px'.format(width)
    width_2 = '{:.0f}px'.format(width/2)
    margin_1 = '2px 2px 2px 2px'
    margin_2 = '1px 1px 5px 1px'
    name_widget = ipywidgets.widgets.ToggleButton(value=False, description=feature_name[feature_index], layout=ipywidgets.Layout(height=height_1, 
width=width_1, margin=margin_1))
    feature_down = ipywidgets.widgets.Button(description='-', layout=ipywidgets.Layout(height=height_1, width=width_2, margin=margin_2))
    feature_up = ipywidgets.widgets.Button(description='+', layout=ipywidgets.Layout(height=height_1, width=width_2, margin=margin_2))
    feature_down.on_click(lambda event: interactive_callback.move_along_axis(event, feature_index, step_size=-0.4))
    feature_up.on_click(lambda event: interactive_callback.move_along_axis(event, feature_index, step_size=0.4))
    sub_widgets = ipywidgets.VBox([name_widget, ipywidgets.Box([feature_down, feature_up])], layout=ipywidgets.Layout(border='1px solid'))
    return sub_widgets

def feature_widgets_layout(widgets_list):
    layout = [[4, 5, 7], [8, 9, 15], [16, 17, 18], [20, 21, 24], [26, 27, 28], [30, 31, 33], [34, 38, 39]]
    return ipywidgets.VBox([ipywidgets.Box([widgets_list[widget] for widget in sub_list]) for sub_list in layout])

widgets_list = []
for feature_index in range(feature_num):
    widgets_list.append(modify_feature_widgets(feature_index))

interactive_callback.set_new_img()
new_img_widget = ipywidgets.widgets.Button(description='random face', layout=ipywidgets.Layout(height='40px', width='128px', margin='1px 1px 5px 1px'))
new_img_widget.on_click(interactive_callback.generate_new_img)
whole_gui = ipywidgets.Box([ipywidgets.VBox([feature_widgets_layout(widgets_list), new_img_widget]), img_widget], layout=ipywidgets.Layout(height='628px', width='1024px'))
display(whole_gui)

Box(children=(VBox(children=(VBox(children=(Box(children=(VBox(children=(ToggleButton(value=False, description…